In [16]:
import pandas as pd
df=pd.read_csv('Bollywood_Songs.csv')
df.head()

,music_name,singer,release,lyrics,thumbnail
0,Ek Haseena Thi Ek Deewana Tha,Yasser Desai,2017,Ek haseena thi ek deewana tha Kya kahun tumse ...,https://i.ytimg.com/vi/6obiArkHwAk/hqdefault.jpg
1,Hue Bechain,"Palak Muchhal, Yasser Desai",2017,Hue bechain pehli bar hmny raaz ye jana Mohabb...,https://i.ytimg.com/vi/6obiArkHwAk/hqdefault.jpg
2,Hanste Hanste,"Palak Muchhal, Yasser Desai",2017,Hanste hanste ro diye tum Kis mushkil mein kho...,https://i.ytimg.com/vi/rVKX19fNV0A/hqdefault.jpg
3,Nain,"Palak Muchhal, Yasser Desai",2017,Kuchh sawaal pyar ke lab pe hai ruke ruke Jaad...,https://i.ytimg.com/vi/ZHbbcy7u9Rk/hqdefault.jpg
4,Aankhon Mein Aansoon,"Palak Muchhal, Yasser Desai",2017,Aankhon mein aansoo leke hoton se muskuraye Aa...,https://i.ytimg.com/vi/izP81UySf0Y/hqdefault.jpg


In [17]:
df=df[['music_name','singer','lyrics']]
df.head()

,music_name,singer,lyrics
0,Ek Haseena Thi Ek Deewana Tha,Yasser Desai,Ek haseena thi ek deewana tha Kya kahun tumse ...
1,Hue Bechain,"Palak Muchhal, Yasser Desai",Hue bechain pehli bar hmny raaz ye jana Mohabb...
2,Hanste Hanste,"Palak Muchhal, Yasser Desai",Hanste hanste ro diye tum Kis mushkil mein kho...
3,Nain,"Palak Muchhal, Yasser Desai",Kuchh sawaal pyar ke lab pe hai ruke ruke Jaad...
4,Aankhon Mein Aansoon,"Palak Muchhal, Yasser Desai",Aankhon mein aansoo leke hoton se muskuraye Aa...


In [18]:
df.shape

(974, 3)

In [19]:
df.isnull().sum()

music_name    0
singer        0
lyrics        0
dtype: int64

In [20]:
df['lyrics'][0]

'Ek haseena thi ek deewana tha Kya kahun tumse kya zamana tha Har jagah toh ye afsana tha Ek haseena thi ek deewana tha  Jab dil toota tha Tum se rootha tha Tum kahan the kahan the kahan the Tum kahan the kahan the kahaaaan  Aansoon girte the tanha firte the Tum kahan the kahan the kahan the Tum kahan the kahan the kahaaaan Tum kahan the kahan the kahan the Tum kahan the kahan the kahaaaan  Khamosh ho gaya tha mera ye jahaan Us waqt humsafar the jane tum kahan Kaise bayaan karun apni majbooriyaan Ek lamhe mein huye milon ki dooriyaan Jab tumko dena mera saath tha Tum kahan the tum kahan the Tum kahan the tum kahan the  Tum kahan the kahan the kahan the Tum kahan the kahan the kahaaaan Tum kahan the kahan the kahan the Tum kahan the kahan the kahaaaan...'

In [21]:
df['lyrics']=df['lyrics'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ', regex=True)
df.head()

,music_name,singer,lyrics
0,Ek Haseena Thi Ek Deewana Tha,Yasser Desai,ek haseena thi ek deewana tha kya kahun tumse ...
1,Hue Bechain,"Palak Muchhal, Yasser Desai",hue bechain pehli bar hmny raaz ye jana mohabb...
2,Hanste Hanste,"Palak Muchhal, Yasser Desai",hanste hanste ro diye tum kis mushkil mein kho...
3,Nain,"Palak Muchhal, Yasser Desai",kuchh sawaal pyar ke lab pe hai ruke ruke jaad...
4,Aankhon Mein Aansoon,"Palak Muchhal, Yasser Desai",aankhon mein aansoo leke hoton se muskuraye aa...


In [22]:
df['lyrics'][0]

'ek haseena thi ek deewana tha kya kahun tumse kya zamana tha har jagah toh ye afsana tha ek haseena thi ek deewana tha  jab dil toota tha tum se rootha tha tum kahan the kahan the kahan the tum kahan the kahan the kahaaaan  aansoon girte the tanha firte the tum kahan the kahan the kahan the tum kahan the kahan the kahaaaan tum kahan the kahan the kahan the tum kahan the kahan the kahaaaan  khamosh ho gaya tha mera ye jahaan us waqt humsafar the jane tum kahan kaise bayaan karun apni majbooriyaan ek lamhe mein huye milon ki dooriyaan jab tumko dena mera saath tha tum kahan the tum kahan the tum kahan the tum kahan the  tum kahan the kahan the kahan the tum kahan the kahan the kahaaaan tum kahan the kahan the kahan the tum kahan the kahan the kahaaaan...'

In [23]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

In [24]:
def token(txt):
    token=nltk.word_tokenize(txt)
    a=[stemmer.stem(w) for w in token]
    return " ".join(a)

In [25]:
token("Playing")

'play'

In [26]:
df['lyrics'] = df['lyrics'].apply(lambda x: token(x))

In [27]:
# Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfid=TfidfVectorizer(analyzer='word', stop_words='english')
matrix=tfid.fit_transform(df['lyrics']) # convert lyrics from songs into numerical vectors

In [28]:
similar=cosine_similarity(matrix) # compares each song vector with every other song

In [29]:
# Recommender Function
def recommender(song_name):
    idx=df[df['music_name']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])), reverse=True, key = lambda x:x[1])
    song=[]
    for s_id in distance[1:6]:
        song.append(df.iloc[s_id[0]].music_name)
    return song

In [30]:
recommender("Nain")

['Beqaaboo', 'Tabaah Ho Gaye', 'Jaan Ban Gaye', 'Hanste Hanste', 'Ghungroo']

In [31]:
import pickle
pickle.dump(similar, open("similarity","wb"))
pickle.dump(df, open("df","wb"))